《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》读书笔记。
 
 [第9章](/2017/07/19/python_data_analysis9.html)  第2节：数据聚合

所有用到的数据可以从[作者的 github](https://github.com/wesm/pydata-book)下载。


In [1]:
%pylab inline
import pandas as pd
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib


GroupBy对象上面优化了一些方法，可以快速进行统计计算，比如：

- count
- sum
- mean
- median（中位数）
- std, var（标准差、方差）
- min, max
- prod （积）
- first, last

但是不仅如此，可以自定义聚合运算。比如， quantile可以计算Series或 DataFrame列的样本分位数：

In [2]:
df = DataFrame({
    'key1': ['a','a','b','b','a'],
    'key2': ['one','two','one','two','one'],
    'data1': np.random.randn(5),
    'data2': np.random.randn(5)
})

df

,data1,data2,key1,key2
0,-0.075862,0.644310,a,one
1,-0.122875,0.692719,a,two
2,0.500193,-0.798486,b,one
3,-1.898311,0.482947,b,two
4,0.221361,-1.521094,a,one


In [3]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.161916
b    0.260343
Name: data1, dtype: float64

如果需要自定义聚合函数，将其传入 aggregate 或 agg 方法即可：

In [4]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.344235,2.213812
b,2.398504,1.281433


In [8]:
# 数据准备
tips = pd.read_csv('data/ch08/tips.csv')

# 增加小费占比（tip_pct）
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


## 面向列的多函数应用

对Series或DataFrame列的聚合运算，其实就是使用aggregate或调用mean, std等方法。

所以很容易对不同的列使用不同的聚合函数，或者一次应用多个函数。

In [11]:
# 根据sex和smoker对tips进行分组
grouped = tips.groupby(['sex', 'smoker'])
grouped_pct = grouped['tip_pct']
# 对于优化过的描述统计，可以直接传入函数名
grouped_pct.agg('mean').unstack()

smoker,No,Yes
sex,,
Female,0.156921,0.182150
Male,0.160669,0.152771


In [13]:
# 传入一组函数/函数名，得到的DataFrame列会自动命名
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [14]:
# 通过 （name, function）元组指定列名
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [15]:
# 对所有列应用一组函数
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [16]:
# 可以指定一组函数的名称
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

In [17]:
# 对不同的列应用不同的函数
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [18]:
# 更复杂的
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                    min       max      mean       std  sum
sex    smoker                                             
Female No      0.056797  0.252672  0.156921  0.036421  140
       Yes     0.056433  0.416667  0.182150  0.071595   74
Male   No      0.071804  0.291990  0.160669  0.041849  263
       Yes     0.035638  0.710345  0.152771  0.090588  150

## 以“无索引”的形式返回聚合数据

参数： as_index=False

In [19]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

,sex,smoker,total_bill,tip,size,tip_pct
0,Female,No,18.105185,2.773519,2.592593,0.156921
1,Female,Yes,17.977879,2.931515,2.242424,0.182150
2,Male,No,19.791237,3.113402,2.711340,0.160669
3,Male,Yes,22.284500,3.051167,2.500000,0.152771


In [20]:
# 对比
tips.groupby(['sex', 'smoker']).mean()

total_bill       tip      size   tip_pct
sex    smoker                                          
Female No       18.105185  2.773519  2.592593  0.156921
       Yes      17.977879  2.931515  2.242424  0.182150
Male   No       19.791237  3.113402  2.711340  0.160669
       Yes      22.284500  3.051167  2.500000  0.152771